In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
#read data
df_directory=r"D:\MY WORK\MaGHALE\bariatric\Data E6.csv"
df=pd.read_csv(df_directory)
df

In [ ]:
#Preprocessing -> Exploring Data
for column in df:
    print(column, df[column].dtype, "{", df[column].unique(),"}", "\n\n")

In [ ]:
#selecting pre-surgery input features and 
# assuming 12-mont and 48 month post-surgery excess weight loss as the outcome
Input_Columnlist_12month=["age", "Height", "WTpre","BMIPpre", "FMpre", 
                  "FBSpre", "a1cpre",
                  "astpre","altpre",
                  "EXTYPEpre", "INTENpre", "MINDAYpre", "SWpre"]
Target1_Column=["EWL12"]
Target2_Column=["EWL48"]

Filter_Columnlist=Target1_Column+Target2_Column+Input_Columnlist_12month
df2=df.filter(items=Filter_Columnlist)
df2

In [ ]:
#Preprocessing -> Exploring Coutns and Missing
for column in df2:
    print(column, "Missing={",df2[column].isnull().sum(),"}", df[column].dtype, "{", df[column].unique(),"}", "\n\n")

In [ ]:
#Preprocessing -> Transforming dummy variables
df3= pd.get_dummies(df2,drop_first=True)
df3

In [ ]:
# exploring data -> scatter plot
pic_list=[]
fig = plt.figure()
for column in df3:
    plt.scatter(df3[column], df3["EWL12"])
    plt.xlabel(str(column))
    plt.show()

# Regression

In [ ]:
# we will create a loop to get results of models using before-surgery features. 
#1-Inputs: All, Target: 12 Month EWL; 2-Inputs: All, Target: 48 Month EWL; 3-Inputs: Selected, Target: 12 Month EWL; 4- Inputs: Selected, Target: 48 Month EWL
# Split the data in 
results=[]
result_Heads=[]

for x in [1,2,3,4]:
    #define input and targets
    X_raw=df3.iloc[:,2:]
    y12_raw=df3.iloc[:,0:1].values
    y48_raw=df3.iloc[:,1:2].values
    
    if x==1:
        plot_head="The cross validaiton results of four regression models. Inputs: All, Target: 12 Month EWL"
        dict_head="InputAll_Target12"
        X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw,y12_raw, test_size=0.1, random_state=14)
        y_raw=y12_raw
        result_head="InputAll_Output12"
    elif x==2:
        plot_head="The cross validaiton results of four regression models. Inputs: All, Target: 48 Month EWL"
        X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw,y48_raw, test_size=0.1, random_state=14)
        y_raw=y48_raw
        dict_head="InputAll_Target48"
        result_head="InputAll_Output48"
    elif x==3:
        plot_head="The cross validaiton results of four regression models. Inputs: Selected, Target: 12 Month EWL"
        X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw,y12_raw, test_size=0.1, random_state=14)
        X_train_selected=X_train.filter(items=["age", "Height", "WTpre", "BMIpre", "FMpre", "MINDAYpre", "SWpre", "InTENpre_Moderate"])
        X_test=X_test.filter(items=["age", "Height", "WTpre", "BMIpre", "FMpre", "MINDAYpre", "SWpre", "InTENpre_Moderate"])
        y_raw=y12_raw
        result_head="InputSelected_Output12"
    elif x==4:
        plot_head="The cross validaiton results of four regression models. Inputs: Selected, Target: 48 Month EWL"
        X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw,y48_raw, test_size=0.1, random_state=14)
        X_train_selected=X_train.filter(items=["age", "Height", "WTpre", "BMIpre", "FMpre", "MINDAYpre", "SWpre", "InTENpre_Moderate"])
        X_test=X_test.filter(items=["age", "Height", "WTpre", "BMIpre", "FMpre", "MINDAYpre", "SWpre", "InTENpre_Moderate"])
        y_raw=y48_raw
        result_head="InputSelected_Output48"
    


    X_train_selected=X_train.filter(items=["age", "Height", "WTpre", "BMIpre", "FMpre", "MINDAYpre", "SWpre", "InTENpre_Moderate"])
    X_test=X_test.filter(items=["age", "Height", "WTpre", "BMIpre", "FMpre", "MINDAYpre", "SWpre", "InTENpre_Moderate"])

    # Preprocessing -> StandardScaler
    scaler = StandardScaler()
    X_train_raw_scaled=scaler.fit_transform(X_train_raw)
    X_test_raw_scaled=scaler.fit_transform(X_test_raw)
    X_raw=scaler.fit_transform(X_raw)
    

    #finding best parameters for GBR using GridSearch
    parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
          'subsample'    : [0.9, 0.5, 0.2, 0.1],
          'n_estimators' : [100,500,1000, 1500],
          'max_depth'    : [4,6,8,10]}
    grid_GBR=GridSearchCV(estimator=GBR, param_grid=parameters, cv=5, n_jobs=-1)
    grid_GBR.fit(X_train_raw_scaled, y_train)
    gbr_score= grid_GBR.best_score_
    gbr_bestparam=grid_GBR.best_params_
    print("for gbr model best score is {} with best parameters {} ".format(gbr_score,gbr_bestparam))

    # getting the cross validation results with best parameters
    gbrmodel=GradientBoostingRegressor(learning_rate= gbr_bestparam["learning_rate"] ,
                                       subsample= gbr_bestparam["subsample"] ,
                                       n_estimators=gbr_bestparam["n_estimators"] ,
                                       max_depth=gbr_bestparam["max_depth"])
    kf = KFold(n_splits=5, random_state=13, shuffle=True)
    gbrcv_results=cross_val_score(gbrmodel, X_raw, y_raw, cv=kf)
    results.append(gbrcv_results)
    result_heading="GBR_"+result_head+ ": Hyperparameters of LR={},subsample={},n_estimator={}, max_depth={}".format(gbr_bestparam["learning_rate"], 
                                                                                                                  gbr_bestparam["subsample"], 
                                                                                                                  gbr_bestparam["n_estimators"],
                                                                                                                  gbr_bestparam["max_depth"])
    
    result_Heads.append(result_heading)


    #finding best alpha parameters for lasso regression
    scores_dict = {}
    for alpha in [0.01, 0.05, 1.0, 2, 3, 4, 10.0, 20.0, 50.0]:
        lasso = Lasso(alpha=alpha)
        lasso.fit(X_train_raw_scaled, y_train)
        lasso_pred = lasso.predict(X_test_raw_scaled)
        score=lasso.score(X_test_raw_scaled, y_test)
        scores_dict[alpha]=score
        print("for alpha of {} the score is {}".format(alpha, score))
    print(scores_dict)
    # getting the cross validation results with best parameters
    selected_lasso = Lasso(alpha=max(scores_dict, key=scores_dict.get))
    kf = KFold(n_splits=5, random_state=13, shuffle=True)
    lassocv_results=cross_val_score(selected_lasso, X_raw, y_raw, cv=kf)
    results.append(lassocv_results)
    result_heading="Lasso_"+result_head+": Hyperparameters of alpha={}".format(max(scores_dict, key=scores_dict.get))
    result_Heads.append(result_heading)
    


    #finding best alpha parameters for rige regression
    scores_dict = {}
    for alpha in [0.01, 0.05, 1.0, 2, 3, 4, 10.0, 20.0, 50.0]:
        ridge = Ridge(alpha=alpha)
        ridge.fit(X_train_raw_scaled, y_train)
        ridge_pred = lasso.predict(X_test_raw_scaled)
        score=ridge.score(X_test_raw_scaled, y_test)
        scores_dict[alpha]=score
        print("for alpha of {} the score is {}".format(alpha, score))
    print(scores_dict)
    # getting the cross validation results with best parameters
    selected_ridge = Ridge(alpha=max(scores_dict, key=scores_dict.get))
    kf = KFold(n_splits=5, random_state=13, shuffle=True)
    ridgcv_results=cross_val_score(selected_ridge, X_raw, y_raw, cv=kf)
    results.append(ridgcv_results)
    result_heading="Ridge_"+result_head+": Hyperparameters of alpha={}".format(max(scores_dict, key=scores_dict.get))
    result_Heads.append(result_heading)


    # The linear regression model
    reg = LinearRegression()
    kf = KFold(n_splits=5, random_state=13, shuffle=True)
    regcv_results=cross_val_score(reg, X_raw, y_raw, cv=kf)
    results.append(regcv_results)
    result_heading="LinearRegression_"+result_head
    result_Heads.append(result_heading)
    pd.DataFrame({"head": result_Heads,"results": results}, columns=['head','results'])

In [ ]:
results_mean=[]
results_std=[]
for i in results:
    print(i)
    results_mean.append(np.mean(i))
    results_std.append(np.std(i))
print(results_mean,results_std)
df_Resultsoutput=pd.DataFrame({"head": result_Heads,"5FoldCV_results": results, "mean": results_mean, "std": results_std}, columns=['head','5FoldCV_results', 'mean', 'std'])
Results_output

In [ ]:
short_head=[]

for i in df_Resultsoutput["head"]:
    seprator= ":"
    short_head.append(i.split(seprator, 1)[0])

df_Resultsoutput["short_head"]=short_head
df_Resultsoutput

In [ ]:
df_boxplot= df_Resultsoutput.drop(["mean","std","head"], axis=1)

CV1=[]
CV2=[]
CV3=[]
CV4=[]
CV5=[]
for i in df_boxplot["5FoldCV_results"]:
    CV1.append(i[0])
    CV2.append(i[1])
    CV3.append(i[2])
    CV4.append(i[3])
    CV5.append(i[4])
df_boxplot["CV1"]=CV1
df_boxplot["CV2"]=CV2
df_boxplot["CV3"]=CV3
df_boxplot["CV4"]=CV4
df_boxplot["CV5"]=CV5
df_boxplot=df_boxplot.set_index("short_head")
df_boxplot=df_boxplot.drop(["5FoldCV_results"], axis=1)
df_boxplot=df_boxplot.T
df_boxplot

In [ ]:
df_boxplot.boxplot( rot=90, fontsize=15)

# Classification

In [ ]:
#read data
df_directory=r"D:\MY WORK\MaGHALE\bariatric\Data E6.csv"
df=pd.read_csv(df_directory)
df